# Homework 05: Decision Trees

* Completar y detallar bien el desarrollo del ejercicio que viene en las filminas que vimos en clase (páginas 33-38 del archivo). Detallen bien paso por paso desarrollando las fórmulas y se pueden apoyar en Excel para realizar las operaciones. 
* También hay que completar y detallar todo el cálculo de las ganancias de las 8 posibles particiones para la discretización binaria del atributo Temperature que viene en las filminas (páginas 48-51 del archivo) y determinar cual es el punto ganador.
* Importante: Todo esto lo deben calcular con los Gains Ratios.

In [27]:
import math

import pandas as pd
import numpy as np

In [4]:
col_names = ['alt', 'bar', 'fri', 'hun', 'pat', 'price', 'rain', 'res', 'type', 'est', 'wait']
df = pd.read_csv('../data/restaurant_wait.csv', names=col_names)
df

,alt,bar,fri,hun,pat,price,rain,res,type,est,wait
0,True,False,False,True,Some,$$$,False,True,French,0-10,True
1,True,False,False,True,Full,$,False,False,Thai,30-60,False
2,False,True,False,False,Some,$,False,False,Burger,0-10,True
3,True,False,True,True,Full,$,False,False,Thai,10-30,True
4,True,False,True,False,Full,$$$,False,True,French,>60,False
5,False,True,False,True,Some,$$,True,True,Italian,0-10,True
6,False,True,False,False,None,$,True,False,Burger,0-10,False
7,False,False,False,True,Some,$$,True,True,Thai,0-10,True
8,False,True,True,False,Full,$,True,False,Burger,>60,False
9,True,True,True,True,Full,$$$,False,True,Italian,10-30,False


In [9]:
df = df.astype({'pat': 'category', 'price': 'category', 'type': 'category', 'est': 'category'})

### Significado de las columnas

* alt : Alternate. Hay restaurantes alternos?
* bar : Bar. Tiene area de bar?
* fri : Fri/Sat. Es viernes o sabado?
* hun : Hungry. Esta uno hambriento?
* pat : Patrons. Cuanta gente hay ya adentro?
* price: Price. Rango de precios del restaurante.
* rain: Rain. Esta lloviendo?
* res: Reservation. Hicimos reservacion o no?
* type: Type. Tipo de comida que sirve el restaurante.
* est: Estimate. Tiempo de espera estimado

* wait: Wait. Esperamos mesa o no?

## Algoritmo DTL

Para este algoritmo tratare de hacerlo lo mas general posible, sin embargo, para una primera version asumire que nuestro valor objetivo siempre es binario (True y False) para simplificar los calculos.

In [85]:
class Node:
    def __init__(self, name, is_leaf=False):
        self.name = name
        self.is_leaf = is_leaf
        self.children = {}
    def __repr__(self):
        if self.children:
            return "{!r}\n{!r}".format(self.name, self.children)
        else:
            return "{!r}\n".format(self.name)

In [86]:
def class_ratios(examples, attribute, value, class_attr):
    counts = examples[examples[attribute] == value][class_attr].value_counts()
    if True not in counts:
        positive = 0
    else:
        positive = counts[True] / sum(counts)
    negative = 1 - positive
    return positive, negative

In [87]:
def H(ratios):
    # To avoid log(0)
    epsilon = 0.000001
    return sum(-1*ratio*math.log(ratio+epsilon, 2) for ratio in ratios)

def choose_attribute(examples, attributes, class_attr):
    counts = examples[class_attr].value_counts()
    initial_info = H((counts[True]/sum(counts), counts[False]/sum(counts)))
    best_gain = 0
    best_attribute = None
    for attribute in attributes:
        counts = examples[attribute].value_counts()
        total = sum(counts)
        split_info = H([count/total for count in counts])
        gain = initial_info
        for value in examples[attribute].unique():
            gain -= len(examples[examples[attribute] == value])/len(examples) * H(class_ratios(examples, attribute, value, class_attr))
        gain_ratio = gain / split_info
        if gain_ratio >= best_gain:
            best_gain = gain_ratio
            best_attribute = attribute
    return best_attribute

In [88]:
def DTL(examples, attributes, class_attribute, default=True):
    if len(examples) == 0:
        return default
    elif len(examples[class_attribute].value_counts()) == 1:
        return examples[class_attribute].unique()[0]
    elif len(attributes) == 0:
        return examples[class_attribute].value_counts().index[0]
    else:
        best_attr = choose_attribute(examples, attributes, class_attribute)
        tree = Node(best_attr, is_leaf=False)
        mode = examples[class_attribute].value_counts().index[0]
        for value in df[best_attr].unique():
            examples_subset = examples[examples[best_attr] == value]
            subtree = DTL(examples_subset, attributes - {best_attr}, class_attribute, mode)
            if not isinstance(subtree, Node):
                subtree = Node(subtree, is_leaf=True)
            tree.children[value] = subtree
        return tree

In [89]:
best_tree = DTL(df, set(df.columns) - {'wait'}, 'wait')

In [90]:
best_tree

'pat'
{'Some': True
, 'Full': 'res'
{True: False
, False: 'fri'
{False: False
, True: 'rain'
{False: True
, True: False
}}}, 'None': False
}